# This notebook investigates using AES

This notebook was just to understand the 16 byte requirements in AES CBC for message padding. Only for reference PyCrypto was used.

### Import necessary modules

In [81]:
from Crypto.Cipher import AES
import Crypto.Util.Padding as pad

import binascii
import os
import struct

### Generate a key

In [82]:
# We want a secure key so generate one with 16 bytes.
key = binascii.hexlify(os.urandom(16))

### Create an initialisation vector

In [83]:
nonce = binascii.hexlify(os.urandom(8)) # AES requires 16 bytes, so urandom creates 8 and hexlify doubles length.

### Create an AES object with the key and IV.

Remember that AES work in blocks of 16 bytes. We will therefore need to do some padding.

##### First have Alice encrypt the message.

In [84]:
# Create the AES object.
aliceAES = AES.new(key, AES.MODE_CBC, nonce) # Cipher Block Chaining mode.

# Create the message. AES works on 16 bytes at a time so the message must be a multiple. We pad it if it is not.
message = b"I want to send a secret message!"
paddedMsg = pad.pad(message, AES.block_size)

# Now call the AES encryption.
cipherText = aliceAES.encrypt(paddedMsg)

print("The cipher text is: {}".format(binascii.hexlify(cipherText)))

The cipher text is: b'1980777811ae5026afa38069296e9a7e846419e43d1fbacc19f0d0559393e147001f1adbedd7438d516736bf4217f3d6'


##### Now have Bob decrypt the message. He recieves the cipher text and the nonce (which could just be appended to the cipher text).

In [85]:
# Assume Bob received the key secretly. He extracts the nonce with his prior knowledge of where to look in the cipher text.
bobAES = AES.new(key, AES.MODE_CBC, nonce)
decryptMsg = bobAES.decrypt(cipherText)

# As there may be some padding from the 16 byte message block size requirement strip off any zeros.
print(pad.unpad(decryptMsg, AES.block_size))

b'I want to send a secret message!'
